# Zero-Coupon Bond Using Marlowe Runtime\'s Command-Line Interface

The zero-coupon bond example is a simple Marlowe contract where a lender provides principal to a borrower who repays it back with interest.

In this demonsration we use Marlowe Runtime\'s command-line interface, `marlowe`, to run this contract on Cardano\'s `preprod` public testnet. Marlowe contracts may use either addresses or role tokens for authorization: here we use addresses.

In [Marlowe Playground](https://play.marlowe-finance.io/), the contract looks like this in Blockly format.

![Zero-coupon bond Marlowe contract](zcb-contract.png)

In Marlowe format it appears as
```
When
    [Case
        (Deposit
            (Address "$LENDER_ADDR")
            (Address "$LENDER_ADDR")
            (Token "" "")
            (ConstantParam "$PRINCIPAL")
        )
        (Pay
            (Address "$LENDER_ADDR")
            (Party (Address "$BORROWER_ADDR"))
            (Token "" "")
            (ConstantParam "$PRINCIPAL")
            (When
                [Case
                    (Deposit
                        (Address "$BORROWER_ADDR")
                        (Address "$BORROWER_ADDR")
                        (Token "" "")
                        (AddValue
                            (ConstantParam "$INTEREST")
                            (ConstantParam "$PRINCIPAL")
                        )
                    )
                    (Pay
                        (Address "$BORROWER_ADDR")
                        (Party (Address "$LENDER_ADDR"))
                        (Token "" "")
                        (AddValue
                            (ConstantParam "$INTEREST")
                            (ConstantParam "$PRINCIPAL")
                        )
                        Close 
                    )]
                (TimeParam "$BORROWER_DEADLINE")
                Close 
            )
        )]
    (TimeParam "$LENDER_DEADLINE")
    Close 
```

## Connecting to Marlowe Runtime

If you are executing this example within [demeter.run](https://demeter.run/), then all of the required environment variables have been set to use Marlowe Runtime on Cardano\'s pre-production network.

If not, the following environment variables must be set to point to the Marlowe Runtime deployment:
- `CARDANO_NODE_SOCKET_PATH`
- `CARDANO_TESTNET_MAGIC`
- `MARLOWE_RT_TX_HOST`
- `MARLOWE_RT_TX_COMMAND_PORT`
- `MARLOWE_RT_HISTORY_HOST`
- `MARLOWE_RT_HISTORY_SYNC_PORT`

Furthermore, the following executables must be on the `PATH`:
- `marlowe`, `marlowe-cli`, and `marlowe-pipe` from [github:input-output-hk/marlowe-cardano](https://github.com/input-output-hk/marlowe-cardano/)
- `cardano-cli` from [github:input-output-hk/cardano-node](https://github.com/input-output-hk/cardano-node/)
- `jq`
- `sed`

## Preliminaries

Set up a few things before working with the contract.

### Constants

Set a few convenient constants for later use.

One ada is one million lovelace, which is the basic unit of currency.

In [1]:
ADA=1000000

In Marlowe, time is measured in milliseconds.

In [2]:
SECOND=1000
MINUTE=$((60 * SECOND))
HOUR=$((60 * MINUTE))
DAY=$((24 * HOUR))

### Create keys and addresses for the parties to the contract

A lender and borrower will interact with the contract. In order to submit transactions, each of them needs a wallet with an address and a signing key.

#### The lender

Set the file names for this party's signing key and verification key.

In [3]:
LENDER_SKEY=lender.skey
LENDER_VKEY=lender.vkey

Generate the keys if they haven't already been generated.

In [4]:
if [[ ! -e "$LENDER_SKEY" ]]
then
  cardano-cli address key-gen \
    --signing-key-file "$LENDER_SKEY" \
    --verification-key-file "$LENDER_VKEY"
fi

Compute the party's address on the testnet.

In [5]:
LENDER_ADDR=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$LENDER_VKEY" )
echo "LENDER_ADDR = $LENDER_ADDR"

LENDER_ADDR = addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy


#### The borrower

Set the file names for this party's signing key and verification key.

In [6]:
BORROWER_SKEY=borrower.skey
BORROWER_VKEY=borrower.vkey

Generate the keys if they haven't already been generated.

In [7]:
if [[ ! -e "$BORROWER_SKEY" ]]
then
  cardano-cli address key-gen \
    --signing-key-file "$BORROWER_SKEY"  \
    --verification-key-file "$BORROWER_VKEY"
fi

Compute the party's address on the testnet.

In [8]:
BORROWER_ADDR=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$BORROWER_VKEY" )
echo "BORROWER_ADDR = $BORROWER_ADDR"

BORROWER_ADDR = addr_test1vryzlnfg9szqzxye7vzzllx5eqsl7u0vas4wnztemha0k5ch2pydf


### Fund the parties' wallets

If the parties do not already have funds in their wallets, send test ada to them. Here are the faucets for the public testnets: https://docs.cardano.org/cardano-testnet/tools/faucet.

It is most convenient to send the funds to Daedalus, and then use Daedalus to fund the three parties. You can download testnet versions of Daedalus at https://docs.cardano.org/cardano-testnet/daedalus-testnet.

However, for this example, we'll use a private faucet to send funds to the three addresses.

In [9]:
marlowe-cli util fund-address \
 --lovelace "$((100 * ADA))" \
 --out-file /dev/null \
 --source-wallet-credentials "$(cat ../faucet.testnet.address):../faucet.skey" \
 --submit 600 \
 "$LENDER_ADDR" "$BORROWER_ADDR" "$GUARANTOR_ADDR"

TxId "500eb161e4bd592518b6d01af91792bb9b639bcc8e4bc2f194d6f7311ffd1ea2"


See that the addresses have indeed been funded:

In [10]:
echo
echo "Lender @ $LENDER_ADDR"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"
echo

echo
echo "Borrower @ $BORROWER_ADDR"
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"
echo


Lender @ addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
500eb161e4bd592518b6d01af91792bb9b639bcc8e4bc2f194d6f7311ffd1ea2     1        100000000 lovelace + TxOutDatumNone


Borrower @ addr_test1vryzlnfg9szqzxye7vzzllx5eqsl7u0vas4wnztemha0k5ch2pydf
                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
500eb161e4bd592518b6d01af91792bb9b639bcc8e4bc2f194d6f7311ffd1ea2     2        100000000 lovelace + TxOutDatumNone



## Design the Contract

The zero-coupon bond contract can be downloaded from the [Marlowe Playground](https://play.marlowe-finance.io/) as a JSON file, or it can be generated using [Marlowe CLI](https://github.com/input-output-hk/marlowe-cardano/tree/main/marlowe-cli#readme) using the `marlowe-cli template` command.

Here we generate the contract using Marlowe CLI.

First find the current time.

In [11]:
NOW="$(($(date -u +%s) * SECOND))"
echo "NOW = $NOW POSIX milliseconds = $(date -d @$((NOW / SECOND)))"

NOW = 1676916039000 POSIX milliseconds = Mon Feb 20 11:00:39 AM MST 2023


Set the principal to 80 ada and the interest to 5 ada.

In [12]:
PRINCIPAL=$((80 * ADA))
INTEREST=$((5 * ADA))
echo "PRINCIPAL = $PRINCIPAL lovelace"
echo "INTEREST = $INTEREST lovelace"

PRINCIPAL = 80000000 lovelace
INTEREST = 5000000 lovelace


The contract has a lending deadline and a repayment deadline. For convenience in this example, set the deadlines to the near future.

In [13]:
LENDER_DEADLINE="$((NOW + 1 * HOUR))"
BORROWER_DEADLINE="$((NOW + 3 * HOUR))"
echo "LENDER_DEADLINE = $LENDER_DEADLINE POSIX milliseconds = $(date -d @$((LENDER_DEADLINE / SECOND)))"
echo "BORROWER_DEADLINE = $BORROWER_DEADLINE POSIX milliseconds = $(date -d @$((BORROWER_DEADLINE / SECOND)))"

LENDER_DEADLINE = 1676919639000 POSIX milliseconds = Mon Feb 20 12:00:39 PM MST 2023
BORROWER_DEADLINE = 1676926839000 POSIX milliseconds = Mon Feb 20 02:00:39 PM MST 2023


On the Cardano blockchain, the protocol parameters require that each UTxO contain at least some ada. Here we will start the contract with 2 ada.

In [14]:
MIN_LOVELACE="$((2 * ADA))"
echo "MIN_LOVELACE = $MIN_LOVELACE lovelace"

MIN_LOVELACE = 2000000 lovelace


Now create the JSON file for the contract.

In [15]:
marlowe-cli template zcb \
  --minimum-ada "$MIN_LOVELACE" \
  --lender "$LENDER_ADDR" \
  --borrower "$BORROWER_ADDR" \
  --principal "$PRINCIPAL" \
  --interest "$INTEREST" \
  --lending-deadline "$LENDER_DEADLINE" \
  --repayment-deadline "$BORROWER_DEADLINE" \
  --out-contract-file zcb-contract.json \
  --out-state-file zcb-state.json

View the contract file as YAML.

In [16]:
json2yaml zcb-contract.json

timeout: 1676919639000
timeout_continuation: close
when:
- case:
    deposits: 80000000
    into_account:
      address: addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy
    of_token:
      currency_symbol: ''
      token_name: ''
    party:
      address: addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy
  then:
    from_account:
      address: addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy
    pay: 80000000
    then:
      timeout: 1676926839000
      timeout_continuation: close
      when:
      - case:
          deposits:
            add: 80000000
            and: 5000000
          into_account:
            address: addr_test1vryzlnfg9szqzxye7vzzllx5eqsl7u0vas4wnztemha0k5ch2pydf
          of_token:
            currency_symbol: ''
            token_name: ''
          party:
            address: addr_test1vryzlnfg9szqzxye7vzzllx5eqsl7u0vas4wnztemha0k5ch2pydf
        then:
          from_account:
            address: addr_test1vryzlnfg9szq

## \[Optional\] Check the safety of the contract

If we were running the contract on the Cardano `mainnet`, then we\'d want to check its safety before creating it, so that there is no chance that we might lose funds.

Here are the steps for checking the safety of a contract:

1. Understand the [Marlowe Language](https://marlowe-finance.io/)
2. Understand Cardano\'s [Extended UTxO Model](https://docs.cardano.org/learn/eutxo-explainer).
3. Read and understand the [Marlowe Best Practices Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/best-practices.md).
4. Read and understand the [Marlowe Security Guide](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/security.md).
5. Use [Marlowe Playground](https://play.marlowe-finance.io/) to flag warnings, perform static analysis, and simulate the contract.
6. Use [Marlowe CLI\'s](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe-cli/ReadMe.md) `marlowe-cli run analyze` tool to study whether the contract can run on a Cardano network.
7. Run *all execution paths* of the contract on a [Cardano testnet](https://docs.cardano.org/cardano-testnet/overview).

Here we\'ll perform step 6. First we bundle the contract and its initial state into a single file.

In [17]:
marlowe-cli run initialize \
  --permanently-without-staking \
  --contract-file zcb-contract.json \
  --state-file zcb-state.json \
  --out-file zcb-marlowe.json

Now we analyze the contract and its execution paths.

In [18]:
marlowe-cli run analyze \
  --marlowe-file zcb-marlowe.json

Note that path-based analysis ignore the initial state of the contract and instead start with an empty state.
Starting search for execution paths . . .
 . . . found 3 execution paths.
- Preconditions:
    Duplicate accounts: []
    Duplicate bound values: []
    Duplicate choices: []
    Invalid account parties: []
    Invalid account tokens: []
    Invalid choice parties: []
    Invalid roles currency: false
    Non-positive account balances: []
- Role names:
    Blank role names: false
    Invalid role names: []
- Tokens:
    Invalid tokens: []
- Maximum value:
    Actual: 88
    Invalid: false
    Maximum: 5000
    Percentage: 1.76
    Unit: byte
- Minimum UTxO:
    Requirement:
      lovelace: 1120600
- Execution cost:
    Memory:
      Actual: 5753324
      Invalid: false
      Maximum: 14000000
      Percentage: 41.095171428571426
    Steps:
      Actual: 1570828622
      Invalid: false
      Maximum: 10000000000
      Percentage: 15.70828622
- Transaction size:
    Actual: 1630


In the above report, we see that the contract doesn\'t have any duplicate or invalid values, and it does not exceed any of the blockchain\'s protocol parameters. In particular, note that our previously chosen `MIN_LOVELACE` value of 2 ada is greater than the 1.120600 ada that the analysis tool says is needed. Thus, it is safe to execute any path in the contract.

## Transaction 1. Create the Contract

Marlowe Runtime\'s command `marlowe create` will build the creation transaction for a Marlowe contract. We provide it the JSON file containing the contract and tell it the `MIN_LOVELACE` value that we previously chose. Anyone could create the contract, but in this example the lender will be doing so, so we provide their address to fund the transaction and to receive the change from it.

In [19]:
CONTRACT_ID=$(
marlowe create \
  --core-file zcb-contract.json \
  --min-utxo "$MIN_LOVELACE" \
  --change-address "$LENDER_ADDR" \
  --manual-sign tx-1.unsigned \
| jq -r 'fromjson | .contractId' \
)
echo "CONTRACT_ID = $CONTRACT_ID"

CONTRACT_ID = ca7adf79d3bb962931aeccacaa5566968c9c5505168bc33d1d3ed03ebf3f5648#1


The result of building the transaction is the identifier for the contract and the file `tx-1.unsigned`, which contains the Cardano unsigned transaction for creating the contract, in text-envelope format.

In [20]:
json2yaml tx-1.unsigned

cborHex: 86a70081825820500eb161e4bd592518b6d01af91792bb9b639bcc8e4bc2f194d6f7311ffd1ea2010d81825820500eb161e4bd592518b6d01af91792bb9b639bcc8e4bc2f194d6f7311ffd1ea2010182a200581d6084b2770ab0d129c619aeedd9be1465abc682735dfaeaad0f5a27e3ed011a05d43683a300581d706a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0011a001e84800282005820ea4ca38ce2978345fae905244c4bf5e7eb0f0859b8a28b6abd948ba4352e7e6010a200581d6084b2770ab0d129c619aeedd9be1465abc682735dfaeaad0f5a27e3ed011a05f12804111a0004b8fc021a000325fd0b58206592d5c1c7768bb66fc0ef240bc05009fd7d7f700698ef8fb31e88f0094ff7c49fff81d8799fd8799f40ffd8799fa1d8799fd8799fd87980d8799fd8799f581c84b2770ab0d129c619aeedd9be1465abc682735dfaeaad0f5a27e3edffd87a80ffffd8799f4040ffff1a001e8480a0a000ffd87c9f9fd8799fd8799fd8799fd87980d8799fd8799f581c84b2770ab0d129c619aeedd9be1465abc682735dfaeaad0f5a27e3edffd87a80ffffd8799fd87980d8799fd8799f581c84b2770ab0d129c619aeedd9be1465abc682735dfaeaad0f5a27e3edffd87a80ffffd8799f4040ffd87a9f1a04c4b400ffffd87a9fd8799fd87980d

There are many ways to sign and submit Cardano transactions:
- `cardano-cli` at the command line
- `cardano-wallet` at the command line or as a REST service
- `cardano-hw-cli` for a hardware wallet at the command line
- a Babbage-compatible CIP-30 wallet in a web browser
- `marlowe-cli` at the command line

For convenience, here we use `marlowe-cli transaction submit`. One may have to wait a minute or so for the transactions to be confirmed on the blockchain.

In [21]:
TX_1_=$(
marlowe-cli transaction submit \
  --tx-body-file tx-1.unsigned \
  --required-signer "$LENDER_SKEY" \
  --timeout 600 \
| sed -e 's/^TxId "\(.*\)"$/\1/' \
)
echo "TX_1 = $TX_1"

TX_1 = ca7adf79d3bb962931aeccacaa5566968c9c5505168bc33d1d3ed03ebf3f5648


One can view the transaction on a Cardano explorer.

In [22]:
echo "https://preprod.cardanoscan.io/transaction/$TX_1?tab=utxo"

https://preprod.cardanoscan.io/transaction/ca7adf79d3bb962931aeccacaa5566968c9c5505168bc33d1d3ed03ebf3f5648?tab=utxo


## View the details of the contract on the blockchain

Marlowe Runtime\'s command `marlowe log` can fetch a contract from the blockchain and print information about it.

In [23]:
marlowe log --show-contract "$CONTRACT_ID"

transaction ca7adf79d3bb962931aeccacaa5566968c9c5505168bc33d1d3ed03ebf3f5648 (creation)
ContractId:      ca7adf79d3bb962931aeccacaa5566968c9c5505168bc33d1d3ed03ebf3f5648#1
SlotNo:          21234813
BlockNo:         650468
BlockId:         54db65b31bfe0f1c7dd33aab569a0ea8c30e63e79aae6062ca9e52b2075bad48
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit (Address "addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy") (Address "addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy")
            (Token "" "")
            (Constant 80000000))
         (Pay (Address "addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy")
            (Party (Address "addr_test1vryzlnfg9szqzxye7vzzllx5eqsl7u0vas4wnztemha0k5ch2pydf"))
            (Token "" "")
            (Constant 80000000)
            (When [
               (Case
                  (Deposit (Address "addr_test1vryzlnfg9sz

More detail can be retrieved using `marlowe-pipe`.

In [24]:
echo '{"request" : "get", "contractId" : "'"$CONTRACT_ID"'"}' | marlowe-pipe 2> /dev/null | json2yaml

creation:
  output:
    address: 706a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0
    assets:
      ada: 2000000
      tokens: []
    datum:
      marloweContract:
        timeout: 1676919639000
        timeout_continuation: close
        when:
        - case:
            deposits: 80000000
            into_account:
              address: addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy
            of_token:
              currency_symbol: ''
              token_name: ''
            party:
              address: addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy
          then:
            from_account:
              address: addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy
            pay: 80000000
            then:
              timeout: 1676926839000
              timeout_continuation: close
              when:
              - case:
                  deposits:
                    add: 80000000
                    and: 5000000
     

## Transaction 2. The lender deposits the principal

The lender deposits their 80 ada of principal into the contract using Marlowe Runtime\'s `marlowe deposit` command. The lender is providing the funding for and receiving the change from this transaction, so we provide their address. We provide the contract identifier and save the unsigned transaction in the file `tx-2.unsigned`.

In [25]:
TX_2=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$LENDER_ADDR" \
  --to-party "$LENDER_ADDR" \
  --lovelace "$PRINCIPAL" \
  --change-address "$LENDER_ADDR" \
  --manual-sign tx-2.unsigned \
| jq -r 'fromjson | .txId' \
)
echo "TX_2 = $TX_2"

TX_2 = 3765960b657280738819afc0b3156a1861124a11efc36485a0d280efe8b473d8


Note that if the transaction would violate the logic of the Marlowe contract, one would receive an error message. For example, let\'s say that we deposit the incorrect amount or deposit it to the wrong party\'s internal account.

In [26]:
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$LENDER_ADDR" \
  --to-party "$LENDER_ADDR" \
  --lovelace 80 \
  --change-address "$LENDER_ADDR" \
  --manual-sign /dev/null

ApplyFailed (ApplyInputsConstraintsBuildupFailed (MarloweComputeTransactionFailed "TEApplyNoMatchError"))


: 1

In [27]:
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$LENDER_ADDR" \
  --to-party "$BORROWER_ADDR" \
  --lovelace "$PRINCIPAL"\
  --change-address "$LENDER_ADDR" \
  --manual-sign /dev/null

ApplyFailed (ApplyInputsConstraintsBuildupFailed (MarloweComputeTransactionFailed "TEApplyNoMatchError"))


: 1

The [Marlowe Debugging Cookbook](https://github.com/input-output-hk/marlowe-cardano/blob/main/marlowe/debugging-cookbook.md) guides interpretation of error messages. Also, one can determine the possible actions for the contract at its current stage of execution by studing the contract\'s current state or by using Marlowe playground to simulate the contract.

![Simulation of zero-coupon bond contract in Marlowe Playground](zcb-simulation.png)

Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [28]:
marlowe-cli transaction submit \
  --tx-body-file tx-2.unsigned \
  --required-signer "$LENDER_SKEY" \
  --timeout 600

TxId "3765960b657280738819afc0b3156a1861124a11efc36485a0d280efe8b473d8"


One can view the transaction on a Cardano explorer.

In [29]:
echo "https://preprod.cardanoscan.io/transaction/$TX_2?tab=utxo"

https://preprod.cardanoscan.io/transaction/3765960b657280738819afc0b3156a1861124a11efc36485a0d280efe8b473d8?tab=utxo


One can see that the lender has 17 ada instead of 100 ada. Two ada were deposited in the contract when it was created and 80 ada were paid to the borrower in the second transaction.

In [30]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
3765960b657280738819afc0b3156a1861124a11efc36485a0d280efe8b473d8     0        17090077 lovelace + TxOutDatumNone


The borrower has the 80 ada of principal now.

In [31]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
3765960b657280738819afc0b3156a1861124a11efc36485a0d280efe8b473d8     2        80000000 lovelace + TxOutDatumNone
500eb161e4bd592518b6d01af91792bb9b639bcc8e4bc2f194d6f7311ffd1ea2     2        100000000 lovelace + TxOutDatumNone


The Marlowe contract still has the 2 ada from its creation.

In [32]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --tx-in "$TX_2#1"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
3765960b657280738819afc0b3156a1861124a11efc36485a0d280efe8b473d8     1        2000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "b72fe7d1e397e9f945e12d543f98193d632d3bcd200cf3d51feca9864c406de3"


## View the further progress of the contract on the blockchain

Marlowe Runtime\'s command `marlowe log` can fetch a contract from the blockchain and print information about it.

In [33]:
marlowe log --show-contract "$CONTRACT_ID"

transaction ca7adf79d3bb962931aeccacaa5566968c9c5505168bc33d1d3ed03ebf3f5648 (creation)
ContractId:      ca7adf79d3bb962931aeccacaa5566968c9c5505168bc33d1d3ed03ebf3f5648#1
SlotNo:          21234813
BlockNo:         650468
BlockId:         54db65b31bfe0f1c7dd33aab569a0ea8c30e63e79aae6062ca9e52b2075bad48
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit (Address "addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy") (Address "addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy")
            (Token "" "")
            (Constant 80000000))
         (Pay (Address "addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy")
            (Party (Address "addr_test1vryzlnfg9szqzxye7vzzllx5eqsl7u0vas4wnztemha0k5ch2pydf"))
            (Token "" "")
            (Constant 80000000)
            (When [
               (Case
                  (Deposit (Address "addr_test1vryzlnfg9sz

More detail can be retrieved using `marlowe-pipe`.

In [34]:
echo '{"request" : "get", "contractId" : "'"$CONTRACT_ID"'"}' | marlowe-pipe 2> /dev/null | json2yaml

creation:
  output:
    address: 706a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0
    assets:
      ada: 2000000
      tokens: []
    datum:
      marloweContract:
        timeout: 1676919639000
        timeout_continuation: close
        when:
        - case:
            deposits: 80000000
            into_account:
              address: addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy
            of_token:
              currency_symbol: ''
              token_name: ''
            party:
              address: addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy
          then:
            from_account:
              address: addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy
            pay: 80000000
            then:
              timeout: 1676926839000
              timeout_continuation: close
              when:
              - case:
                  deposits:
                    add: 80000000
                    and: 5000000
     

## Transaction 3. The borrower repays the loan

After some time passes, the borrower repays principal plus interest. Thus, they fund the transaction and receive the change at their address.

In [35]:
TX_3=$(
marlowe deposit \
  --contract "$CONTRACT_ID" \
  --from-party "$BORROWER_ADDR" \
  --to-party "$BORROWER_ADDR" \
  --lovelace "$((PRINCIPAL+INTEREST))" \
  --change-address "$BORROWER_ADDR" \
  --manual-sign tx-3.unsigned \
| jq -r 'fromjson | .txId' \
)
echo "TX_3 = $TX_3"

TX_3 = 30b58c5cd0cb52ba16139a3f4026970b5da0c820d13e7b36e68628f33a529580


Once again, use `marlowe-cli` to submit the transaction and then wait for confirmation.

In [36]:
marlowe-cli transaction submit \
  --tx-body-file tx-3.unsigned \
  --required-signer "$BORROWER_SKEY" \
  --timeout 600

TxId "30b58c5cd0cb52ba16139a3f4026970b5da0c820d13e7b36e68628f33a529580"


One can view the transaction on a Cardano explorer.

In [37]:
echo "https://preprod.cardanoscan.io/transaction/$TX_3?tab=utxo"

https://preprod.cardanoscan.io/transaction/30b58c5cd0cb52ba16139a3f4026970b5da0c820d13e7b36e68628f33a529580?tab=utxo


One can see that the lender received back the 80 ada of principal and the 2 ada deposited when the contract was created, along with the additional 5 ada of interest, totallying 87 ada.

In [39]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$LENDER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
30b58c5cd0cb52ba16139a3f4026970b5da0c820d13e7b36e68628f33a529580     1        87000000 lovelace + TxOutDatumNone
3765960b657280738819afc0b3156a1861124a11efc36485a0d280efe8b473d8     0        17090077 lovelace + TxOutDatumNone


The borrower now has a total of about 95 ada, which is 100 ada minus the 5 ada paid as interest.

In [39]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$BORROWER_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
30b58c5cd0cb52ba16139a3f4026970b5da0c820d13e7b36e68628f33a529580     0        14441267 lovelace + TxOutDatumNone
3765960b657280738819afc0b3156a1861124a11efc36485a0d280efe8b473d8     2        80000000 lovelace + TxOutDatumNone


The Marlowe contract has closed, so there is no output to its script address.

## View the completion of the contract on the blockchain

Marlowe Runtime\'s command `marlowe log` can fetch a contract from the blockchain and print information about it.

In [40]:
marlowe log --show-contract "$CONTRACT_ID"

transaction ca7adf79d3bb962931aeccacaa5566968c9c5505168bc33d1d3ed03ebf3f5648 (creation)
ContractId:      ca7adf79d3bb962931aeccacaa5566968c9c5505168bc33d1d3ed03ebf3f5648#1
SlotNo:          21234813
BlockNo:         650468
BlockId:         54db65b31bfe0f1c7dd33aab569a0ea8c30e63e79aae6062ca9e52b2075bad48
ScriptAddress:   addr_test1wp4f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvqu3c6jv
Marlowe Version: 1

    When [
      (Case
         (Deposit (Address "addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy") (Address "addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy")
            (Token "" "")
            (Constant 80000000))
         (Pay (Address "addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy")
            (Party (Address "addr_test1vryzlnfg9szqzxye7vzzllx5eqsl7u0vas4wnztemha0k5ch2pydf"))
            (Token "" "")
            (Constant 80000000)
            (When [
               (Case
                  (Deposit (Address "addr_test1vryzlnfg9sz

More detail can be retrieved using `marlowe-pipe`.

In [41]:
echo '{"request" : "get", "contractId" : "'"$CONTRACT_ID"'"}' | marlowe-pipe 2> /dev/null | json2yaml

creation:
  output:
    address: 706a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0
    assets:
      ada: 2000000
      tokens: []
    datum:
      marloweContract:
        timeout: 1676919639000
        timeout_continuation: close
        when:
        - case:
            deposits: 80000000
            into_account:
              address: addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy
            of_token:
              currency_symbol: ''
              token_name: ''
            party:
              address: addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy
          then:
            from_account:
              address: addr_test1vzztyac2krgjn3se4mkan0s5vk4udqnnthaw4tg0tgn78mgnhrkyy
            pay: 80000000
            then:
              timeout: 1676926839000
              timeout_continuation: close
              when:
              - case:
                  deposits:
                    add: 80000000
                    and: 5000000
     

## \[Optional\] Clean Up

We can tidy up the wallets by sending their funds back to the original faucet.

In [42]:
marlowe-cli transaction simple \
  --tx-in "$TX_2#0" \
  --tx-in "$TX_2#2" \
  --tx-in "$TX_3#0" \
  --tx-in "$TX_3#1" \
  --change-address "$(cat ../faucet.testnet.address)" \
  --required-signer "$LENDER_SKEY" \
  --required-signer "$BORROWER_SKEY" \
  --out-file /dev/null \
  --submit 600

TxId "80052388722e75613371368cafc531cce2bd1e505c6870674de3610bd4bdb8ac"
